# TODO:
    - head
    - info
    - data dict (types, e.g. nominal, categorial)
    - matrix (missing values)
    - bar (missing values)
    - box (numeric, deviation)
    - bar/mosaic/ (categorial, deviation)
    - predictor/feature correlation (heatmap/scatter)
    - histogram (skewed/deviation)


In [1]:
import idstools.data_explorer as idsde

In [2]:
data = "../data/BikeRentalDaily_train.csv"

In [3]:
data_explorer_config = {"input_file": {"path": data, "type": "csv", "sep": ";"}, "output_path": "../results/"}

In [4]:
data_explorer = idsde.data_explorer(data_explorer_config)

In [5]:
data_explorer.descriptive_analysis()
data_explorer.description

,count,mean,std,min,25%,50%,75%,max
instant,600.0,363.12,208.71,1.00,181.25,362.50,538.25,731.00
season,538.0,2.44,1.11,1.00,1.00,2.00,3.00,4.00
yr,600.0,0.50,0.50,0.00,0.00,0.00,1.00,1.00
mnth,600.0,6.47,3.44,1.00,4.00,6.00,9.00,12.00
holiday,600.0,0.03,0.17,0.00,0.00,0.00,0.00,1.00
weekday,600.0,2.93,2.08,-1.00,1.00,3.00,5.00,6.00
workingday,600.0,0.68,0.47,0.00,0.00,1.00,1.00,1.00
weathersit,600.0,1.40,0.54,1.00,1.00,1.00,2.00,3.00
temp,600.0,19.81,7.21,2.37,13.57,20.10,26.06,34.47
atemp,600.0,0.47,0.16,0.08,0.34,0.49,0.61,0.83


Wir starten mit der diskriptiven Analyse der Daten, dafür verwenden wir discrbe Funktion der Pandas Library.

In [6]:
import idstools.data_preparation as dp

In [7]:
test_data = "/home/davidrmn/Studies/introduction-data-science/data/BikeRentalDaily_test.csv"
train_data = "/home/davidrmn/Studies/introduction-data-science/data/BikeRentalDaily_train.csv"

In [8]:
data_preparation_config = {
    "input_file": {
        "path": train_data,
        "type": "csv",
        "sep": ";"
    },
    "output_path": None,
}

In [9]:
data_preparation = dp.data_preparation(config=data_preparation_config)


In [10]:
import time
import pandas as pd

def get_wday_by_date(df, date_column, weekday_column):
    # Define the weekday shift
    weekday_shift = {
        6: 0,
        0: 1,
        1: 2,
        2: 3,
        3: 4,
        4: 5,
        5: 6
    }

    # Convert the date column to datetime
    df[date_column] = pd.to_datetime(df[date_column], format="%d.%m.%Y")

    # Calculate the weekday and map it
    df[weekday_column] = df[date_column].dt.dayofweek.map(weekday_shift)

    return df

In [11]:
pipeline_config = {
        "Imputer": [
            {
                "target": "hum",
                "config": {
                    "strategy": "mean"
                }
            }
        ],
        "GenericDataFrameTransformer": [
        {
            "transform_func": get_wday_by_date,
            "config": {
                "data_column": "dteday",
                "weekday_column": "weekday"
            }
        }
    ]
}

            {
                "target": "dteday",
                "config": {
                    "source": "module",
                    "module": "pandas",
                    "function": "to_datetime",
                    "args": {
                        "format": "%d.%m.%Y"
                    },
                    "axis": 1
                }
            },

In [12]:
pipeline = data_preparation.build_pipeline(config=pipeline_config)
pipeline

Pipeline(steps=[('Imputer',
                 Imputer(config=[{'config': {'strategy': 'mean'},
                                  'target': 'hum'}])),
                ('OneHotEncoder', None), ('FeatureDropper', None),
                ('GenericDataFrameTransformer',
                 GenericDataFrameTransformer(config=[{'config': {'data_column': 'dteday',
                                                                 'weekday_column': 'weekday'},
                                                      'transform_func': 'get_wday_by_date'}]))])

In [13]:
processed_data = data_preparation.run_pipeline(config=pipeline_config)

In [14]:
processed_data.head(5).T

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
processed_data.describe().T